In [13]:
import ess_premium_analysis_3wayadjustments
import bbgclient
import datetime
import dfutils
import pandas as pd
import ast

In [14]:
#Necessary input items from the database
alpha = "TOWR US EQUITY"
CIX_Index = ".ESSTOWR INDEX"
unaffected_date = "2018-11-19"
tgt_date = "2019-03-08"
exp_close_date = "2020-03-31"

analyst_upside = 24.7944386560876
analyst_downside = 11.8720668879134
analyst_pt_wic = 22.78075067876
peers_names = ["MGA US EQUITY", "MRE CN EQUITY", "DAN US EQUITY"]
peers_weights = [0.4,0.4,0.2]
multiples_names = ["EV/Sales", "EV/EBITDA", "P/EPS", "DVD yield", "FCF yield"]
multiples_weights = [0,0.75,0,0,0.25]
p = None

#The functions as for the api_host and 
api_host = bbgclient.bbgclient.get_next_available_host()
#Create a dictionary of the peers and weights and the multiples and weights

def create_dict(keys, values):
    return dict(zip(keys, values + [None] * (len(keys) - len(values))))

#Take the peers vs. weights and the multiples vs. weights and create dictionaries
#These two factors must have type dictionary for input
peersVSweights = create_dict(peers_names, peers_weights)
multiplesVSweights = create_dict(multiples_names, multiples_weights)
as_of_dt = datetime.datetime.today()
unaff_dt = datetime.datetime.strptime(unaffected_date, '%Y-%m-%d')
tgt_dt = datetime.datetime.strptime(tgt_date, '%Y-%m-%d')

In [15]:
adj_df_now = '[{"Date":"2019-03-08","PX":"0","BEST_EPS":"0","BEST_NET_INCOME":"0","BEST_OPP":"0","BEST_SALES":"0","CUR_EV_COMPONENT":"-250","CUR_MKT_CAP":"0","DIVIDEND_INDICATED_YIELD":"0","BEST_CAPEX":"0","BEST_EBITDA":"0","EQY_SH_OUT":"0"}]'

#x = ast.literal_eval(adj_df_now)[0]
#adjustments_df_new = pd.DataFrame.from_dict(x, orient = 'index')
#adjustments_df_new = adjustments_df_new.T
#adjustments_df_new = adjustments_df_new.drop(columns = 'Date')
#cols = adjustments_df_new.columns
#adjustments_df_new[cols] = adjustments_df_new[cols].apply(pd.to_numeric)

In [16]:
adj_df_ptd = '[{"Date":"2019-03-08","PX":"0","BEST_EPS":"0","BEST_NET_INCOME":"0","BEST_OPP":"0","BEST_SALES":"0","CUR_EV_COMPONENT":"-250","CUR_MKT_CAP":"0","DIVIDEND_INDICATED_YIELD":"0","BEST_CAPEX":"0","BEST_EBITDA":"0","EQY_SH_OUT":"0"}]'
#x = ast.literal_eval(adj_df_ptd)[0]
#adjustments_df_new = pd.DataFrame.from_dict(x, orient = 'index')
#adjustments_df_new = adjustments_df_new.T
#adjustments_df_new = adjustments_df_new.drop(columns = 'Date')
#cols = adjustments_df_new.columns
#adjustments_df_new[cols] = adjustments_df_new[cols].apply(pd.to_numeric)
#adjustments_df_new['CUR_EV_COMPONENT'][0]

In [17]:
#Function that takes input and runs the ess_premium_analysis.py document
#The output of this function is the rsults of each model as dataframes
def run_ess_premium_analysis(alpha_ticker, unaffectedDt, tgtDate, as_of_dt, analyst_upside,analyst_downside, analyst_pt_wic, peers2weight, metric2weight, api_host, adjustments_df_now = None, adjustments_df_ptd = None, premium_as_percent = None, f_period = '1BF'):
    slicer = dfutils.df_slicer()
    start_date = slicer.prev_n_business_days(120, unaffectedDt) #lookback is 120 days (6 months)
    metrics = {k: v for k, v in metric2weight.items() if v != 0}
    peers_list = list(peers2weight.keys())
    metric_list = list(metrics.keys())

    calib_data = ess_premium_analysis_3wayadjustments.calibration_data(alpha_ticker, peers2weight, start_date, unaffectedDt, metric_list, api_host, f_period)

    OLS_results = ess_premium_analysis_3wayadjustments.premium_analysis_df_OLS(alpha_ticker, peers_list, calib_data, analyst_upside, analyst_downside, analyst_pt_wic, as_of_dt, tgtDate, metric_list, metric2weight, api_host, adjustments_df_now, adjustments_df_ptd, premium_as_percent)
    premium_analysis_results = ess_premium_analysis_3wayadjustments.premium_analysis_df(alpha_ticker, peers_list, as_of_dt, tgtDate, analyst_upside, analyst_downside, analyst_pt_wic, metric_list, metric2weight, calib_data['metric2rel'], api_host, adjustments_df_now, adjustments_df_ptd, premium_as_percent)

    return(premium_analysis_results, OLS_results)

In [18]:
model1, model2 = run_ess_premium_analysis(alpha, unaff_dt, tgt_dt, as_of_dt, analyst_upside, analyst_downside, analyst_pt_wic, peersVSweights, multiplesVSweights, api_host, adjustments_df_now = None, adjustments_df_ptd = None, premium_as_percent = None, f_period = '1BF')
model2

,Metric,OLS HTML,MGA US EQUITY,MRE CN EQUITY,DAN US EQUITY,Intercept,Peers Multiples DataFrame @ Price Target Date,Alpha Implied Multiple @ Price Target Date,Alpha Balance Sheet DataFrame @ Price Target Date,Alpha Unaffected PX @ Price Target Date,...,Alpha Unaffected PX @ Now,Alpha Upside (analyst),Alpha Downside (analyst),Alpha PT WIC (analyst),Premium Up ($),Premium PT WIC ($),Premium Down ($),"Alpha Downside (Adj,weighted)","Alpha PT WIC (Adj,weighted)","Alpha Upside (Adj,weighted)"
0,EV/EBITDA,"<table class=""simpletable"">\n<caption>OLS Regr...",-0.038861,0.292706,0.466788,1.062431,Peer Multiple 0 MGA US EQUITY ...,3.621919,Date PX CUR_MKT_CAP EQY_SH_OUT B...,19.594759,...,19.939117,24.794439,11.872067,22.780751,5.199680,3.185992,-7.722692,9.162319,17.343832,18.854098
1,FCF yield,"<table class=""simpletable"">\n<caption>OLS Regr...",-0.170054,0.314420,0.103736,0.050322,Peer Multiple 0 MGA US EQUITY 0...,0.072729,Date PX CUR_MKT_CAP EQY_SH_OUT B...,127.662143,...,128.834600,24.794439,11.872067,22.780751,-102.867704,-104.881392,-115.790076,3.261131,5.988302,6.491724


In [11]:
model2['Alpha Implied Multiple @ Now'][1]

0.0723596926060244

In [12]:
model1

,Metric,Alpha to Peer historical ratio (mean),Alpha to Peer historical ratio (std),Peers Composite Multiple @ Price Target Date,Peer2Multiple @ Price Target Date,Alpha Implied Multiple (mean) @ Price Target Date,Alpha Balance Sheet @ Price Target Date,Alpha Unaffected PX @ Price Target Date,Alpha Upside (analyst),Alpha Downside (analyst),...,Alpha Unaffected PX (high) @ Now,Premium Up ($),Premium PT WIC ($),Premium Down ($),Alpha Downside (Adj),Alpha Upside (Adj),Alpha PT WIC (Adj),"Alpha Downside (Adj,weighted)","Alpha PT WIC (Adj,weighted)","Alpha Upside (Adj,weighted)"
0,EV/EBITDA,0.907183,0.082406,4.03540,"{'MGA US EQUITY': 4.719, 'MRE CN EQUITY': 3.54...",3.660848,Date PX CUR_MKT_CAP EQY_SH_OUT B...,19.925584,24.794439,11.872067,...,25.944769,4.868854,2.855166,-8.053517,12.182854,25.105225,23.091537,9.137140,17.318653,18.828919
1,FCF yield,0.731310,0.109216,0.11444,"{'MGA US EQUITY': 0.12144856437645962, 'MRE CN...",0.083691,Date PX CUR_MKT_CAP EQY_SH_OUT B...,110.939595,24.794439,11.872067,...,160.443127,-86.145156,-88.158844,-99.067528,13.453312,26.375683,24.361995,3.363328,6.090499,6.593921


In [19]:
def final_df(alpha_ticker, cix_index, unaffectedDt, expected_close, tgtDate, analyst_upside, analyst_downside, analyst_pt_wic, peers2weight, metric2weight, api_host, adjustments_df_now = None, adjustments_df_ptd = None, premium_as_percent = None, f_period = "1BF"):
    slicer = dfutils.df_slicer()
    as_of_dt = datetime.datetime.today()
    unaff_dt = datetime.datetime.strptime(unaffectedDt, '%Y-%m-%d')
    tgt_dt = datetime.datetime.strptime(tgtDate, '%Y-%m-%d')
    exp_close_dt = datetime.datetime.strptime(expected_close, '%Y-%m-%d')
    
    df = pd.Series()
    df["Alpha Ticker"] = alpha_ticker
    df["Unaffected Date"] = unaff_dt.strftime("%Y-%m-%d")
    df["CIX"] = cix_index
    df["Expected Close Date"] = exp_close_dt.strftime('%Y-%m-%d')
    
    df["Alpha Downside"] = analyst_downside
    df["Alpha PT (WIC)"] = analyst_pt_wic
    df["Alpha Upside"] = analyst_upside
    
    df["CIX at Target Date"] = bbgclient.bbgclient.get_timeseries(cix_index,'PX_LAST', slicer.prev_n_business_days(100,tgt_dt).strftime('%Y%m%d'), tgt_dt.strftime('%Y%m%d'), api_host=api_host).iloc[-1]
    df["Alpha at Target Date"] = bbgclient.bbgclient.get_timeseries(alpha_ticker,'PX_LAST', slicer.prev_n_business_days(100,tgt_dt).strftime('%Y%m%d'), tgt_dt.strftime('%Y%m%d'), api_host=api_host).iloc[-1]
    df["Alpha Last Price"] = bbgclient.bbgclient.get_timeseries(alpha_ticker,'PX_LAST', slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'), as_of_dt.strftime('%Y%m%d'), api_host=api_host).iloc[-1]
    df["CIX Last Price"] = bbgclient.bbgclient.get_timeseries(cix_index,'PX_LAST', slicer.prev_n_business_days(100,as_of_dt).strftime('%Y%m%d'), as_of_dt.strftime('%Y%m%d'),api_host=api_host).iloc[-1]
    
    df["CIX Upside"] = (df["Alpha Upside"] - df["Alpha at Target Date"]) + df["CIX at Target Date"]
    df["CIX Downside"] = df["CIX at Target Date"] - (df["Alpha at Target Date"] - df["Alpha Downside"])
    df["CIX WIC Adjustment"] = (df["Alpha PT (WIC)"] - df["Alpha at Target Date"]) + df["CIX at Target Date"]
    df["Down Price (CIX)"] = df["Alpha Last Price"] - (df["CIX Last Price"] - df["CIX Downside"])
    df["Up Price (CIX)"] = df["Alpha Last Price"] + (df["CIX Upside"] - df["CIX Last Price"])
    df["PT WIC Price (CIX)"] = df["Alpha Last Price"] + (df["CIX WIC Adjustment"] - df["CIX Last Price"])
    
    #try:
    model1, model2 = run_ess_premium_analysis(alpha_ticker, unaff_dt, tgt_dt, as_of_dt, analyst_upside, analyst_downside, analyst_pt_wic, peers2weight, metric2weight, api_host, adjustments_df_now, adjustments_df_ptd, premium_as_percent, f_period = "1BF")
    #except Exception as e:
        #print('failed running WIC and Regression models: ' + str(e.args))
    
    df["Down Price (WIC)"] = model1["Alpha Downside (Adj,weighted)"].sum()
    df["Down Price (Regression)"] = model2["Alpha Downside (Adj,weighted)"].sum()
    df["PT WIC Price (WIC)"] = model1["Alpha PT WIC (Adj,weighted)"].sum()
    df["PT WIC Price (Regression)"] = model2["Alpha PT WIC (Adj,weighted)"].sum()
    df["Up Price (WIC)"] = model1["Alpha Upside (Adj,weighted)"].sum()
    df["Up Price (Regression)"] = model2["Alpha Upside (Adj,weighted)"].sum()
    
    df["Embedded Premium($)"] = df["Alpha Last Price"] - ((df["Down Price (WIC)"] + df["Up Price (WIC)"])/2.0)
    df["Embedded Premium(%)"] = 100.0*(df["Embedded Premium($)"]/df["Alpha Last Price"])
    
    df["Probability (CIX)"] = 100.0*((df["CIX Last Price"] - df["CIX Downside"])/(df["CIX Upside"] - df["CIX Downside"]))
    df["Probability (Alpha)"] = 100.0*((df["Alpha Last Price"] - df["Alpha Downside"])/(df["Alpha Upside"] - df["Alpha Downside"]))
    if pd.isna(df['Probability (CIX)']):
        df['Probability(%)'] = df['Probability (Alpha)']
    else:
        df['Probability(%)'] = df['Probability (CIX)']
    #df['Probability(%)'] = [prob_cix if not pd.isnull(prob_cix) else prob_alpha for (prob_cix, prob_alpha) in zip(df["Probability (CIX)"], df["Probability (Alpha)"])]
    
    df["Return/Risk (CIX)"] = (df["CIX Upside"] - df["CIX Last Price"])/(df["CIX Last Price"] - df["CIX Downside"])
    df["Return/Risk (Alpha)"] = (df["Alpha Upside"] - df["Alpha Last Price"])/(df["Alpha Last Price"] - df["Alpha Downside"])
    if not pd.isna(df['Return/Risk (CIX)']):
        df['Return/Risk'] = df['Return/Risk (CIX)']
    else:
        df['Return/Risk'] = df['Probability (Alpha)']
    #df["Return/Risk"] = [radj_cix if not pd.isnull(radj_cix) else radj_alpha for (radj_cix, radj_alpha) in zip(df["Return/Risk (CIX)"], df["Return/Risk (Alpha)"])]
    
    df["Gross Return (CIX)"] = 100.0*((df["CIX Upside"] - df["CIX Last Price"])/df["Alpha Last Price"])
    df["Gross Return (Alpha)"] = 100.0*((df["Alpha Upside"] - df["Alpha Last Price"])/df["Alpha Last Price"])
    if not pd.isna(df['Gross Return (CIX)']):
        df['Gross Return(%)'] = df['Gross Return (CIX)']
        gross = df['Gross Return(%)']
    else:
        df['Gross Return(%)'] = df['Gross Return (Alpha)']
        gross = df['Gross Return(%)']
    #df['Gross Return(%)'] = [gross_ret_cix if not pd.isnull(gross_ret_cix) else gross_ret_alpha for (gross_ret_cix, gross_ret_alpha) in zip(df["Gross Return (CIX)"], df["Gross Return (Alpha)"])]

    #d1 = datetime.datetime.strptime(expected_close, '%Y-%m-%d')
    #d1 = datetime.datetime.strptime(as_of_dt, '%Y-%m-%d')
    days = abs((exp_close_dt - as_of_dt).days)
    df["Days to Close"] = [None if pd.isnull(days) else days]
    if (pd.isna(df['Gross Return(%)']) or pd.isna(df["Days to Close"]) or days <= 7):
        df['Ann. Return(%)'] = None
    else:
        df['Ann. Return(%)'] = 100.0*((1+gross/100)**(365.0/days)-1.0)
    #df['Ann. Return(%)'] = [None if (pd.isnull(ret) or pd.isnull(days) or days <= 7) else 100.0*((1+ret/100)**(365.0/days)-1.0) for (ret, days) in zip(df['Gross Return(%)'] , df["Days to Close"])]                
    
    
    float_cols = ["Alpha Upside",
                "Alpha Downside",
                "Alpha Last Price",
                'Up Price (CIX)',
                'Down Price (CIX)',
                'CIX Last Price',
                'Return/Risk',
                'Probability(%)',
                'Gross Return(%)',
                'Ann. Return(%)',
                'Up Price (WIC)',
                'Down Price (WIC)',
                'Up Price (Regression)',
                'Down Price (Regression)',
                'Embedded Premium($)',
                'Embedded Premium(%)']
    df[float_cols] = df[float_cols].astype(float)
    
    cols2show = ['Alpha Ticker',
                 'Unaffected Date',
                 'Expected Close Date',
                 'CIX',
                 'Alpha Last Price',
                 'CIX Last Price',
                 'Alpha Downside',
                 'Down Price (CIX)',
                 'Down Price (WIC)',
                 'Down Price (Regression)',
                 'Alpha PT (WIC)',
                 'PT WIC Price (CIX)',
                 'PT WIC Price (WIC)',
                 'PT WIC Price (Regression)',
                 'Alpha Upside',
                 'Up Price (CIX)',
                 'Up Price (WIC)',
                 'Up Price (Regression)',
                 'Embedded Premium($)',
                 'Embedded Premium(%)',
                 'Probability(%)',
                 'Return/Risk',
                 'Gross Return(%)',
                 'Ann. Return(%)']
    
    return(df[cols2show])

In [20]:
final_df(alpha, CIX_Index, unaffected_date, exp_close_date, tgt_date, analyst_upside, analyst_downside, analyst_pt_wic, peersVSweights, multiplesVSweights, api_host, adjustments_df_now = adj_df_now, adjustments_df_ptd = adj_df_ptd, premium_as_percent = p, f_period = "1BF")

Alpha Ticker                 TOWR US EQUITY
Unaffected Date                  2018-11-19
Expected Close Date              2020-03-31
CIX                          .ESSTOWR INDEX
Alpha Last Price                      23.88
CIX Last Price                      -0.7488
Alpha Downside                      11.8721
Down Price (CIX)                     12.098
Down Price (WIC)                    12.3101
Down Price (Regression)              12.371
Alpha PT (WIC)                      22.7808
PT WIC Price (CIX)                  23.0067
PT WIC Price (WIC)                  23.2188
PT WIC Price (Regression)           23.2797
Alpha Upside                        24.7944
Up Price (CIX)                      25.0203
Up Price (WIC)                      25.2324
Up Price (Regression)               25.2934
Embedded Premium($)                 5.10874
Embedded Premium(%)                 21.3934
Probability(%)                      91.1755
Return/Risk                       0.0967862
Gross Return(%)                 